# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    12-Jan-2026**

**News Source / Catalyst:** "US prosecutors launch criminal investigation into Federal Reserve’s Jay Powell" - Claire Jones in London and Kate Duguid in New York

**Markets (s) Impacted:** FX, EQ, COM, FI (but, trade will be executed via FX and COM)

**Trade Taxonomy:** RV Macro / Structural FX

**Trade Idea (1 sentence):**
> The Trump administration's litigation against the Fed, commands a short dollar trade against a basket of European currencies and safe-haven assets.

**Client/Desk Pitch (30 seconds):**
> The trade monetises institutional credibility risk. If markets believe the White House is influencing the Fed, the USD should weaken, real yields should rise via term premium, and safe-haven alternatives (EUR/CHF/Gold) should catch a bid.

**Why Now:**
- The is the first wave of litigation by the White House against the Fed, allowing this trade to crossover from thematic into event-driven
- There is limited geopolitical risk felt amongst our European currency-basket, when juxtaposed to the US, making this trade FX fundamentally driven
- Asia has already shown to be pricing in this risk with gold hitting USD 4,600 a troy ounce and other bullions rallying, all prior to US and Europe-market opening, creating slight-information asymmetry to be traded on

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
- The value of USD will trade down against their European peers, out of fears driving by a lack of separation between the government and central bank
- This lack of value will increase the risk premium found amongst UST, USD, inflating US yields
- Asia reacted first via bullion, Europe opens second via EUR crosses, NY comes last with USD repricing
- Taking a longer view, as Powell is stepping down in May, and will likely be replaced by a more dovish peer, EQ investors will hold off in overly-participating in trade due to the increased likelihood of lower rates in the future, and FI investors will reduce increase US new issue participation on the likelihood of locking in higher rates prior, to the Fed's May pivot

**Positioning (Tickers + Rationale):**
- EUR/USD (EURUSD=X), GBP/USD (GBPUSD=X), CHF/USD (CHFUSD=X), Gold (GC=F)

    **Weights:** Equal-Weights
---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |          S        |       |
| Oil Beta             |          W        |       |
| Rates Duration       |           S       |       |
| Growth vs Value      |             W     |       |
| Risk-On/Off          |            M      |       |
| China/EM Beta        |             W     |       |
| Volatility Regime    |            S      |       |

**Catalyst Type:** Policy + Regime + Event Driven

**Realisation Window:** 1-week (to take quick volatility driven profit, however could be extended depending on client's manadate)  

**Holding Period Selected:**  10-trading days (as the White House is expected to announce their chairman nomination in January 2026)

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** N/A
**Stop-Loss:** (portfolio-level %) 5% (0.05)  
**Take-Profit (optional):**  3% (0.03)
**Re-entry:** Yes/No  
**FX-Hedging Required:** No  
**Liquidity Notes:** N/A

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [ ] Information

---


# **PYTHON IMPLEMENTATION**

In [47]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [48]:
# --- Parameters ---

long_tickers = ["EURUSD=X", "GBPUSD=X", "GC=F"] 
short_tickers = ["USDCHF=X"] # Using this ticker to reduce python errors                   

start_date = "2020-01-01"                        
holding_period = 10                              
stop_loss = -0.05                                
take_profit = 0.02                              

# --- Weightings ---

weights_long = 1 / len(long_tickers)
weights_short = 1 / len(short_tickers)


In [49]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_35607/911793947.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  4 of 4 completed


In [50]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [51]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
151,2025-10-21,-0.0504
152,2025-11-04,0.0248
153,2025-11-18,0.0039
154,2025-12-02,0.0232
155,2025-12-16,0.0229


In [52]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         156.0000
Win Rate               0.5705
Average PnL            0.0019
Best Trade             0.0719
Worst Trade           -0.0699
Stop-Out Frequency     0.0705
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 156.0
- Win Rate: 0.5705
- Avg PnL: 0.0019
- Best: 0.0719
- Worst: -0.0699
- Stop-Outs: 0.0705

**What went right:**
- 

**What went wrong:**
- 

**Factor that actually drove results:**
- 

**Would I trade this live?** (Yes / No and why)
>

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [53]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [54]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [55]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
151,RV-61D2C432,2026-01-12 07:31:36,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, GC=F",USDCHF=X,2025-10-21,2025-11-03,1362.6092,1333.4505,-5.0392,-5.0000,2.0000,STOPPED,13,
152,RV-B3D9389B,2026-01-12 07:31:36,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, GC=F",USDCHF=X,2025-11-04,2025-11-17,1315.9137,1356.1316,2.2401,-5.0000,2.0000,TAKE_PROFIT,13,
153,RV-212CDB54,2026-01-12 07:31:36,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, GC=F",USDCHF=X,2025-11-18,2025-12-01,1353.7953,1413.1246,0.3876,-5.0000,2.0000,CLOSED,13,
154,RV-044D1677,2026-01-12 07:31:36,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, GC=F",USDCHF=X,2025-12-02,2025-12-15,1395.5560,1435.6074,2.1341,-5.0000,2.0000,TAKE_PROFIT,13,
155,RV-845B53D6,2026-01-12 07:31:36,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, GC=F",USDCHF=X,2025-12-16,2025-12-30,1434.8752,1456.7541,1.9174,-5.0000,2.0000,CLOSED,14,


In [56]:
blotter.to_csv("trade_blotter.csv", index=False)